In [43]:
from typing import Iterable

import numpy as np

from nupack import Complex as NupackComplex
from nupack import Model as NupackModel
from nupack import ComplexSet as NupackComplexSet
from nupack import Strand as NupackStrand
from nupack import SetSpec as NupackSetSpec
from nupack import complex_analysis as nupack_complex_analysis
from nupack import PairsMatrix as NupackPairsMatrix


import dsd.vienna_nupack as dv
import dsd.constraints as dc

# constants
DOMAIN_LENGTH = 15
DOMAIN_POOL: dc.DomainPool = dc.DomainPool('DOMAIN_POOL', DOMAIN_LENGTH)
TEMPERATURE = 37.0
NUPACK_MODEL = NupackModel(material='dna', celsius=TEMPERATURE)
NUMBER_OF_TRIALS = 100

def nupack_base_pair_probabilities(strands: Iterable[dc.Strand], strands_to_assign: Iterable[dc.Strand], base_index1: int, base_index2: int) -> np.ndarray:
    """Given a design and a specific base pair, assigns random DNA sequences to
    the design and computes the equilibrium base-pairing probability for each
    DNA sequence assignment.
    
    :param strands: The strands that make up the design.
    :type strands: Iterable[dc.Strand]
    :param strands_to_assign: The strands to assign DNA to. Domains
        complementary to these strands are automatically reassigned DNA. As
        such, strands_to_assign is usually a subset of strands.
    :type strands: Iterabble[dc.Strand]
    :param base_index1: The index of one of the bases that form the base pair
    :type base_index1: int
    :param base_index2: The index of the other base that forms the base pair
    :type base_index2: int

    :return: An array of NUMBER_OF_TRIALS base pair probabilities
    :rtype: numpy.ndarray
    """
    base_pair_probabilities = [0] * NUMBER_OF_TRIALS
    for i in range(NUMBER_OF_TRIALS):
        # Assign random DNA sequence
        for s in strands_to_assign:
            s.assign_dna(dv.random_dna_seq(s.length()))
            
        # Initialize NUPACK NupackComplexSet
        nupack_strands = [NupackStrand(strand.sequence(), name=strand.name) for strand in strands]
        nupack_complex: NupackComplex = NupackComplex(nupack_strands)
        nupack_complex_set = NupackComplexSet(nupack_strands, complexes=(nupack_complex,))        
        
        # Call NUPACK complex_analysis        
        nupack_complex_result: np.ndarray = nupack_complex_analysis(nupack_complex_set, compute=['pairs'], model=NUPACK_MODEL)[nupack_complex].pairs.to_array()
            
        # Collect results
        base_pair_probabilities[i] = nupack_complex_result[base_index1][base_index2]
        
    return np.array(base_pair_probabilities)

In [44]:
print('INTERIOR_TO_STRAND')
# INTERIOR_TO_STRAND
#                       a      b
#                     0  14 15  29
#                     |   |  |   |
#                    [-----##----->
#                     |||||  |||||
#                    <-----##-----]
#                     |   |  |   |
#                     59 45  44  30
#                       a*^   b*
#                         |
#                     base pair
top_strand: dc.Strand = dc.Strand(['a', 'b'], name='top strand')
bot_strand: dc.Strand = dc.Strand(['b*', 'a*'], name='bot strand')

for s in [top_strand, bot_strand]:
    for d in s.domains:
        d.pool = DOMAIN_POOL

interior_to_strand_bpps = nupack_base_pair_probabilities((top_strand, bot_strand), (top_strand,), 14, 45)
print('bpps:', base_pair_probabilities)
print('mean', np.mean(base_pair_probabilities))
print('standard deviation:', np.std(base_pair_probabilities))


INTERIOR_TO_STRAND
bpps: [0.99816101 0.99972153 0.99617988 0.99925856 0.99523222 0.98509368
 0.99871567 0.99696114 0.99757056 0.99869996 0.99941051 0.99869997
 0.98502704 0.99256159 0.99769141 0.99417769 0.99694625 0.99898727
 0.99816052 0.99236872 0.99759692 0.9975826  0.99926089 0.99876037
 0.99815973 0.99543612 0.99814155 0.99696701 0.99509437 0.99236629
 0.99522968 0.99925364 0.99545992 0.99619751 0.99693481 0.99769347
 0.99971841 0.999002   0.99417876 0.99507806 0.99041929 0.99947327
 0.99815987 0.99256157 0.9976525  0.99769392 0.99547297 0.99925382
 0.99869103 0.99256137 0.99077116 0.99815819 0.99256142 0.99814193
 0.9975652  0.99544268 0.99045848 0.99696264 0.98193123 0.99871746
 0.99769083 0.9990278  0.99696179 0.99814468 0.99814158 0.99756448
 0.99947934 0.99237071 0.99875958 0.99761493 0.99928034 0.99547289
 0.99695528 0.99946896 0.99769389 0.99260259 0.99755609 0.99900648
 0.99947962 0.99871806 0.99755595 0.99619716 0.99039332 0.99931144
 0.99968486 0.99814195 0.99693921 0.9